# The official tutorial for OmniXAI NLP explainer

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICE'] = '1'

In [2]:
import numpy as np
import transformers
from omnixai.data.text import Text
from omnixai.explainers.nlp import NLPExplainer

/home/kxlu/anaconda3/envs/omnixai/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-27 19:31:44.379709: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-27 19:31:44.430854: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-27 19:31:45.353171: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could n

The test input

In [39]:
x = Text([
    "What a great movie!",
    "The Interview was neither that funny nor that witty. "
    "Even if there are words like funny and witty, the overall structure is a negative type."
])

Define the pipeline and the NLPExplainer object

In [40]:
# The preprocessing function
preprocess = lambda x: x.values
# A transformer model for sentiment analysis
model = transformers.pipeline(
    'sentiment-analysis',
    model='distilbert-base-uncased-finetuned-sst-2-english',
    return_all_scores=True
)
# The postprocessing function
postprocess = lambda outputs: np.array([[s["score"] for s in ss] for ss in outputs])

# Initialize a NLPExplainer
explainer = NLPExplainer(
    explainers=["shap", "lime"],
    mode="classification",
    model=model,
    preprocess=preprocess,
    postprocess=postprocess
)

/home/kxlu/anaconda3/envs/omnixai/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:104: UserWarning:

`return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.



Skip SHAP and LIME as it only supports classification.

Skip L2X as it requires traning data.

In [8]:
# Generates explanations
local_explanations = explainer.explain(x)

print("SHAP results:")
local_explanations["shap"].ipython_plot()

SHAP results:


In [9]:
print("LIME results:")
local_explanations["lime"].ipython_plot()

LIME results:


Polyjuice has errors.

In [41]:
explainer = NLPExplainer(
    explainers=["polyjuice", "lime"],
    mode="classification",
    model=model,
    preprocess=preprocess,
    postprocess=postprocess
)
local_explanations = explainer.explain(x)
print("Counterfactual results:")
local_explanations.ipython_plot()

INFO:polyjuice.polyjuice_wrapper:Setup Polyjuice.
INFO:polyjuice.polyjuice_wrapper:Setup SpaCy processor.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


ValueError: Explainer polyjuice -- `early_stopping` must be a boolean or 'never', but is None.